In [ ]:
x=1

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import random
import scanpy as sc
import numpy as np
import pandas as pd
import muon as mu
import torch

In [ ]:
def set_seed(seed):
    """设置所有相关的随机种子，以确保实验可复现"""
    random.seed(seed)  
    np.random.seed(seed)  
    torch.manual_seed(seed) 
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False 

set_seed(42)

In [ ]:
rna = sc.read('./data/neurips-multiome/rna_hvg.h5ad')
atac = sc.read('./data/neurips-multiome/atac_hvf.h5ad')

mdata = mu.MuData({'rna': rna, 'atac': atac})

mdata.obs['batch'] = mdata['rna'].obs['batch'].astype('category').copy()
mdata.obs['cell_type'] = mdata['rna'].obs['cell_type'].astype('category').copy()

atac.X = atac.layers['cpm'].copy()

mu.tl.mofa(
    mdata, 
    groups_label='batch',
    gpu_mode = False,
)

rna.obsm['latent'] = mdata.obsm['X_mofa']
rna.write('./results/neurips-multiome-mofa.h5ad')

In [ ]:
rna = sc.read('./data/neurips-cite/rna_hvg.h5ad')
adt = sc.read('./data/neurips-cite/protein.h5ad')

adt.var_names = ['PROT_'+name for name in adt.var_names]
rna.var_names = ['RNA_'+name for name in rna.var_names]

mdata = mu.MuData({'rna': rna, 'adt': adt})

mdata.obs['batch'] = mdata['rna'].obs['batch'].astype('category').copy()
mdata.obs['cell_type'] = mdata['rna'].obs['cell_type'].astype('category').copy()

mu.tl.mofa(mdata, groups_label='batch', gpu_mode=True)

rna.obsm['latent'] = mdata.obsm['X_mofa']
rna.write('./results/neurips-cite-mofa.h5ad')